# Notes

Things to keep in mind while reading this consolidated code:
1. This is consolidated code to understand the *workflow* of this project from start to end
2. In this code, you will be able to follow all the *major steps*
3. I will *skip the detailed data patching/cleaning*. There are invalid data from google maps (e.g. two entries in google maps for the same mall)
4. I will *refer to a specific notebook that I used* and needed files (if any) when creating this project for every section

# 1. *Nilai Jual Objek Pajak* (NJOP) download and parse

**Needed file**:
njop2021.pdf from [Pergub Nomor 17 Tahun 2021 Tentang Penetapan NJOP PBB-P2 Tahun 2021](https://bprd.jakarta.go.id/peraturan-perpajakan/unduh/pergub-nomor-17-tahun-2021-tentang-penetapan-njop-pbbp2-tahun-2021)

**Relevant Notebook File**:
transform_njop_to_csv.ipynb

**Output files**:
1. njop2021.pdf
2. NJOP2021.csv


## 1.1. Download the NJOP File

In [3]:
import requests
url = 'https://bprd.jakarta.go.id/peraturan-perpajakan/unduh/pergub-nomor-17-tahun-2021-tentang-penetapan-njop-pbbp2-tahun-2021'
response = requests.get(url)
with open('njop2021.pdf', 'wb') as f:
    f.write(response.content)

## 1.2. Parse the NJOP File

In [ ]:
from PyPDF2 import PdfReader
import numpy as np
import pandas as pd
import re

provinsi = []
kota = []
kecamatan = []
kelurahan = []
blk = []
nama_jalan = []
znt = []
kelas_bumi = []
min_njb = []
sd = []
max_njb = []
ave_njb = []
page_num = []

reader = PdfReader("njop2021.pdf")

# parsing by pattern
for num in range(4, 4204): # until page 4203
    print("Now on page", num)
    page = reader.pages[num]
    x = page.extract_text()
    x = x.splitlines()
    words = [word for word in x if not '$' in word]
    for i in range(len(words)):
        # every page contains only one provinsi, kota, kecamatan, and kelurahan
        # next to every title is the information
        if 'PROPINSI' in words[i-1]:
            provinsi_page = words[i]
        if 'KOTA/KAB' in words[i-1]:
            kota_page = words[i]
        if 'KECAMATAN' in words[i-1]:
            kecamatan_page = words[i]
        if 'KELURAHAN' in words[i-1]:
            kelurahan_page = words[i]
    for i in range(len(words)):
        """
        Pattern:
        1. 'BLK' contains 3 numbers
        2. 'ZNT code' contains 2 letters
        3. 'Kelas Bumi' contains 3 numbers
        4. 'Sixth column' is 's/d' string
        """
        # let's use last column as the ith place
        if re.match(r"\d{3}", x[i-7])\
        and re.match(r"[A-Z]{2}", x[i-5])\
        and re.match(r"\d{3}", x[i-4])\
        and x[i-2] == 's/d':
            provinsi.append(provinsi_page)
            kota.append(kota_page)
            kecamatan.append(kecamatan_page)
            kelurahan.append(kelurahan_page)
            blk.append(x[i-7])
            nama_jalan.append(x[i-6])
            znt.append(x[i-5])
            kelas_bumi.append(x[i-4])
            min_njb.append(x[i-3])
            sd.append(x[i-2])
            max_njb.append(x[i-1])
            ave_njb.append(x[i])
            page_num.append(num)

# insert to a DataFrame
df = pd.DataFrame(data={'provinsi': provinsi
                  , 'kota': kota
                  , 'kecamatan': kecamatan
                  , 'kelurahan': kelurahan
                  , 'blk': blk
                  , 'nama_jalan': nama_jalan
                  , 'znt': znt
                  , 'kelas_bumi': kelas_bumi
                  , 'min_njb': min_njb
                  , 'sd': sd
                  , 'max_njb': max_njb
                  , 'ave_njb': ave_njb
                  , 'page_num': page_num})

# create CSV file from parsed NJOP
df.to_csv('NJOP2021.csv')

# 2. Scrape from [Lamudi](https://www.lamudi.co.id/)

**Relevant Notebook File**: scrape_lamudi.ipynb

**Output file**: lamudi_house_dataset.csv

In [ ]:
import numpy as np
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import math

links = []
data_price_s = []
data_category_s = []
data_subcategories_s = []
data_bedrooms_s = []
data_bathrooms_s = []
data_building_size_s = []
data_land_size_s = []
data_furnished_s = []
data_sku_s = []
data_geo_point_s = []
page_link_s = []
parent_link_s = []

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.get("https://www.lamudi.co.id/jakarta/house/buy/")
click_expand = driver.find_element(By.XPATH, '//*[@id="js-crosslinkTopFilter"]/a').click()
sublinks = driver.find_elements(By.XPATH, '//*[@id="js-crosslinkTopFilter"]/div/div/div/a')
for tag in sublinks:
    links.append(tag.get_attribute('href'))
driver.quit()
for link in links:
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(link)
    page_end = 1 # initialize page_end with number 1 to be refined, only for looping purpose
    page_iter = 1
    while page_iter <= page_end:
        info = driver.find_elements(By.CSS_SELECTOR, ".ListingCell-AllInfo.ListingUnit")
        info_a = driver.find_elements(By.CSS_SELECTOR, ".js-listing-link")
        try:
            page_end = int(driver.find_element(By.CSS_SELECTOR, "*[data-pagination-end]:last-child").get_attribute('data-pagination-end'))
        except:
            pass
        for i in range(len(info)):
            data_price_s.append(info[i].get_attribute('data-price'))
            data_category_s.append(info[i].get_attribute('data-category'))
            data_subcategories_s.append(info[i].get_attribute('data-subcategories'))
            data_bedrooms_s.append(info[i].get_attribute('data-bedrooms'))
            data_bathrooms_s.append(info[i].get_attribute('data-bathrooms'))
            data_building_size_s.append(info[i].get_attribute('data-building_size'))
            data_land_size_s.append(info[i].get_attribute('data-land_size'))
            data_furnished_s.append(info[i].get_attribute('data-furnished'))
            data_sku_s.append(info[i].get_attribute('data-sku'))
            data_geo_point_s.append(info[i].get_attribute('data-geo-point'))
            page_link_s.append(info_a[i*2].get_attribute('href')) # js-listing-link class is written twice in the website
            parent_link_s.append(link)
        page_iter += 1
        if page_end >= page_iter:
            click_expand = driver.find_element(By.CSS_SELECTOR, '.next a[data-next-page]').click()
            time.sleep(math.ceil(2.811 + np.random.normal(loc=0.0,scale=0.627), 0)) # delay so it doesn't break or startle the website
    driver.quit()

# insert to a DataFrame
df = pd.DataFrame(data={'data_price_s': data_price_s,
'data_category_s': data_category_s,
'data_subcategories_s': data_subcategories_s,
'data_bedrooms_s': data_bedrooms_s,
'data_bathrooms_s': data_bathrooms_s,
'data_building_size_s': data_building_size_s,
'data_land_size_s': data_land_size_s,
'data_furnished_s': data_furnished_s,
'data_sku_s': data_sku_s,
'data_geo_point_s': data_geo_point_s,
'page_link_s': page_link_s,
'parent_link_s': parent_link_s})

# create CSV file from scraped website
df.to_csv('lamudi_house_dataset.csv')

# 3. Geocode from NJOP Files

**Relevant Notebook Files**:
1. NJOP2021_geocode.ipynb
2. EDA_NJOP.ipynb

**Output File**: jakarta_street_coordinate_patched.csv

In [ ]:
from geopy.geocoders import Nominatim # https://nominatim.org/
import numpy as np
import pandas as pd
import random
import time

df = pd.read_csv('./NJOP2021.csv')
df = df.drop(['Unnamed: 0', 'blk', 'znt', 'kelas_bumi', 'sd', 'page_num'], axis=1)

cols = ['provinsi', 'kota', 'kecamatan', 'kelurahan']
for col in cols:
    df[col] = df[col].apply(lambda x: str(x).split(' - ')[1])

# There are weird kecamatan and kelurahan from NJOP Files to be cleaned
remove_spaces = [
    'P U L O', 'C I K O K O', 'P A N C O R A N', 'R A G U N A N', 'S E L O N G', 'S E N A Y A N', 'K A R E T'
    , 'G U N T U R', 'KALI BATA', 'B A N G K A', 'U L U J A M I', 'C I P U L I R', 'PAL MERIAM', 'KALI ANYAR'
    , 'RAWA JATI', 'SETIA BUDI'
]
df['kelurahan'] = df['kelurahan'].apply(lambda x: x.replace(' ', '') if x in remove_spaces else x)
df['kelurahan'] = df['kelurahan'].str.replace('KEBAYORAN LAMA UTR', 'KEBAYORAN LAMA UTARA').str.replace('KEBAYORAN LAMA SLT', 'KEBAYORAN LAMA SELATAN').str.replace('BALIMESTER', 'BALI MESTER')
remove_spaces = ['C I L A N D A K', 'J A G A K A R S A', 'P A N C O R A N', 'T E B E T', 'SETIA BUDI']
df['kecamatan'] = df['kecamatan'].apply(lambda x: x.replace(' ', '') if x in remove_spaces else x)
df = df.drop_duplicates(ignore_index=True)

dict = {}
for idx in range(len(df)):
    print('Now on index', idx)
    try: # try except is needed since my internet connection is unstable
        geolocator = Nominatim(user_agent="yusuf-application")
        location = geolocator.geocode(df.iloc[idx]['nama_jalan']+', '+df.iloc[idx]['kelurahan'], timeout=10)
        if location:
            dict[idx] = [location.latitude, location.longitude]
    except:
        idx -= 1
        time.sleep(60) # wait for internet to reconnect
for i in dict:
    df['coordinate'].iloc[i] = dict[i]
df = df.dropna().reset_index(drop=True)
# create CSV file from street and coordinate
df.to_csv('jakarta_street_coordinate_patched.csv')

# 4. Exploratory Data Analysis (EDA) Lamudi

**Input File**: lamudi_house_dataset.csv

**Relevant Notebook File**: EDA_NJOP.ipynb

**Output File**: lamudi_house_dataset_cleaned.csv

In [ ]:
# for processing dataset
import numpy as np
import pandas as pd
import re
from math import ceil, floor

# for plotting
import matplotlib.pyplot as plt
import seaborn as sns

# to display all the columns of the dataframe in the notebook
pd.pandas.set_option('display.max_columns', None)

df = pd.read_csv('lamudi_house_dataset.csv')
df = df.drop('Unnamed: 0', axis=1) # from the creation of csv
# remove duplicates
df = df.drop_duplicates('data_sku_s', keep='first').reset_index(drop=True)
print('Number of rows after dropping duplicate rows:', len(df))
# remove incomplete important data
df = df[ ~df['data_price_s'].isna() ].reset_index(drop=True)
df = df[ ~df['data_geo_point_s'].isna() ].reset_index(drop=True)
# remove outliers
min_bathroom_s = 1
min_bedroom_s = 1
min_building_size_s = 6 # I guess 6 meters-squared is already a valid size
min_land_size_s = 6 # The same with land size
df = df[ df['data_bathrooms_s'] >= min_bathroom_s ]
df = df[ df['data_bedrooms_s'] >= min_bedroom_s ]
df = df[ df['data_building_size_s'] >= min_building_size_s ]
df = df[ df['data_land_size_s'] >= min_land_size_s ]
max_bathroom_s = np.percentile(df['data_bathrooms_s'], 99)
max_bedroom_s = np.percentile(df['data_bedrooms_s'], 99)
max_building_size_s = np.percentile(df['data_building_size_s'], 99)
max_land_size_s = np.percentile(df['data_land_size_s'], 99)
df = df[ df['data_bathrooms_s'] <= max_bathroom_s ]
df = df[ df['data_bedrooms_s'] <= max_bedroom_s ]
df = df[ df['data_building_size_s'] <= max_building_size_s ]
df = df[ df['data_land_size_s'] <= max_land_size_s ]
df = df.reset_index(drop=True)
print('Number of rows after cleaning:', len(df))

# create kabupaten and kecamatan columns
kabupaten_pattern = r'([-\w]+)/([-\w]+)/house/buy/$'
df['kabupaten'] = df['parent_link_s'].apply(lambda x: re.search(kabupaten_pattern, x)[1])
kecamatan_pattern = r'/([-\w]+)/house/buy/$'
df['kecamatan'] = df['parent_link_s'].apply(lambda x: re.search(kecamatan_pattern, x)[1])

# create CSV file from cleaned lamudi house dataset
df.to_csv('lamudi_house_dataset_cleaned.csv')

# 5. Scrape Google Maps for Facilities

**Relevant Notebook File**: scrape_gmaps.ipynb

**Output File**: facility_latlong.csv

*Notes*: I skipped many manual patchings in this consolidated file to make it easier to read. Data from Google Maps in Indonesia is not as clean and we still need to patch the data manually

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import requests
import re
import numpy as np
import pandas as pd
import time
import random
import math

# malls
malls = []
driver = webdriver.Chrome()
driver.get("https://id.wikipedia.org/wiki/Daftar_pusat_perbelanjaan_di_Jakarta")
mall_s = driver.find_elements(By.XPATH, '//*[@id="mw-content-text"]/div[1]/ul/li')
for mall in mall_s:
    malls.append(mall.text)
driver.quit()
malls = [mall.replace('Mall', 'Mal').replace('Mal', 'Mall') for mall in malls]

# train stations
krls = []
driver = webdriver.Chrome()
driver.get("https://id.wikipedia.org/wiki/Kategori:Stasiun_kereta_api_di_Jakarta")
krl_s = driver.find_elements(By.XPATH, '//*[@id="mw-pages"]/div/div/div/ul/li/a')
for krl in krl_s:
    krls.append(krl.text)
driver.quit()

# transjakarta (bus)
tjs = []
driver = webdriver.Chrome()
driver.get("https://id.wikipedia.org/wiki/Daftar_koridor_Transjakarta")
tj_s = driver.find_elements(By.CSS_SELECTOR, 'tr>td:nth-child(2)>a.mw-redirect[href^="/wiki/"]')
for tj in tj_s:
    tjs.append('Halte Busway ' + tj.text)
driver.quit()
tjs = sorted(set(tjs))

# hospitals
rss = []
driver = webdriver.Chrome()
driver.get("https://id.wikipedia.org/wiki/Daftar_rumah_sakit_di_DKI_Jakarta")
rs_s = driver.find_elements(By.CSS_SELECTOR, 'tr>td:nth-child(3)>a.new[href]')
for rs in rs_s:
    rss.append(rs.text)
driver.quit()

list_to_scrape = malls + krls + tjs + rss
len(list_to_scrape)

url_s = ['']*len(list_to_scrape)
stars = ['']*len(list_to_scrape)
count_reviews = ['']*len(list_to_scrape)
category = ['mall']*len(malls) + ['train_station']*len(krls) + ['bus_station']*len(tjs) + ['hospital']*len(rss)

# scrape from google maps
driver = webdriver.Chrome()
for i in range(len(list_to_scrape)):
    driver.get("https://www.google.com/maps/@-6.208428,106.7824432,15z")
    time.sleep(math.ceil(1.81 + np.random.normal(loc=0.0,scale=0.327), 0))
    Place = driver.find_element(By.CLASS_NAME, "tactile-searchbox-input")
    Place.send_keys(list_to_scrape[i])
    time.sleep(math.ceil(1.81 + np.random.normal(loc=0.0,scale=0.327), 0))
    Place.send_keys(Keys.DOWN)
    time.sleep(0.15)
    Submit = driver.find_element(By.ID, "searchbox-searchbutton")
    Submit.click()
    time.sleep(math.ceil(1.81 + np.random.normal(loc=0.0,scale=0.127), 0))
    try:
        url_s[i] = driver.current_url
        stars[i] = driver.find_element(By.CSS_SELECTOR, 'span[aria-label^="Bintang"]').accessible_name # bintang = star
        count_reviews[i] = driver.find_element(By.CSS_SELECTOR, 'span[aria-label$="lasan"]').accessible_name # ulasan = review
    except:
        pass

df = pd.DataFrame(data = {
    'name': list_to_scrape
    , 'url': url_s
    , 'stars': stars
    , 'count_reviews': count_reviews
    , 'category': category
})

# clean scraping data
df['coordinate'] = df['url'].apply(lambda x: re.search(r'@([-\d\.,]+)z', x)[1])
df['stars'] = df['stars'].apply(lambda x: re.search(r'^Bintang\s([\d,]+)', str(x))[1] if re.search(r'^Bintang\s([\d,]+)', str(x)) else None)
df['count_reviews'] = df['count_reviews'].apply(lambda x: re.search(r'([\d\.]+)\s.lasan$', str(x))[1] if re.search(r'([\d\.]+)\s.lasan$', str(x)) else None)
df['stars'] = df['stars'].str.replace(',', '.').astype('float64')
df['stars'] = df['stars'].fillna(3)
df['count_reviews'] = df['count_reviews'].fillna('5')
df['count_reviews'] = df['count_reviews'].str.replace('.', '').astype(int)

# split to get latitude and longitude
df['latitude'] = df['coordinate'].apply(lambda x: x.split(',')[0])
df['longitude'] = df['coordinate'].apply(lambda x: x.split(',')[1])

# create facilities and latitude
df.to_csv('facility_latlong.csv', index=False)

# 6. Add Jakarta Flood Data from [Governmental Dashboard](https://public.tableau.com/app/profile/jsc.data/viz/DataPendukungPotensiGenangan/PetaAwal)

**Input Files**:
1. flood_0.png
2. flood_1.png
3. flood_2.png
4. flood_3.png
5. flood_4.png
6. flood_5.png
7. flood_6.png

**Relevant Notebook File**: check_flood.ipynb

*Notes*: Since the mapping from pixel to coordinate is tied to the pixel of the screen, it is better to just copy from my screenshot

In [ ]:
from PIL import Image

def get_pixel(lat, long):
    return [ int(round(160+(478-160)*(long-(106.7351))/(106.91427-((106.7351))))), int(round(145+(654-145)*(lat-(-6.0808))/(-6.36549-((-6.0808))))) ]

def get_flood_scenario(lat, long):
    colors_list = []
    pixel = get_pixel(lat, long)
    flood_0 = 0
    flood_1 = 0
    flood_2 = 0
    flood_3 = 0
    flood_4 = 0
    flood_5 = 0
    flood_6 = 0
    for i in range(7):
        im = Image.open(f"./flood_{i}.png")
        rgb_im = im.convert('RGB')
        for h_px in range(-3,4):
            for v_px in range(-3,4):
                try:
                    colors_list.append( rgb_im.getpixel((pixel[0]+v_px, pixel[1]+h_px)) )
                except:
                    pass
        if i == 0 and (225, 87, 89) in colors_list:
            flood_0 = 1
        if i == 1 and (237, 201, 72) in colors_list:
            flood_1 = 1
        if i == 2 and (78, 121, 167) in colors_list:
            flood_2 = 1
        if i == 3 and (242, 142, 43) in colors_list:
            flood_3 = 1
        if i == 4 and (89, 161, 79) in colors_list:
            flood_4 = 1
        if i == 5 and (176, 122, 161) in colors_list:
            flood_5 = 1
        if i == 6 and (156, 117, 95) in colors_list:
            flood_6 = 1
    return [flood_0, flood_1, flood_2, flood_3, flood_4, flood_5, flood_6]

# 7. Merge Data from Lamudi, NJOP, Flood Scenarios, and Facilities

**Detailed inputs, notebooks, and outputs in the subsections**

## 7.1. Merge Data from Lamudi and NJOP

**Input Files**:
1. lamudi_house_dataset_cleaned.csv
2. jakarta_street_coordinate_patched.csv

**Relevant Notebook File**: EDA_NJOP.ipynb

**Output File**: lamudi_njop_dataset.csv

In [ ]:
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from geopy.geocoders import Nominatim # https://nominatim.org/
import time

df_njop = pd.read_csv('NJOP2021.csv')
njop_cols = ['kelurahan', 'nama_jalan', 'ave_njb']
df_njop = df_njop[njop_cols]
df_njop['kelurahan'] = df_njop['kelurahan'].apply(lambda x: str(x).split(' - ')[1])

df = pd.read_csv('jakarta_street_coordinate_patched.csv')
df_map = df[cols].merge(df_njop, how='inner', on=['kelurahan', 'nama_jalan'])
df_map['ave_njb'] = df_map['ave_njb'].str.replace(',', '').astype('float')

# https://stackoverflow.com/questions/1253499/simple-calculations-for-working-with-lat-lon-and-km-distance
# Latitude: 1 deg = 110.574 km
# Longitude: 1 deg = 111.320*cos(latitude) km
def get_average_njop_near(lat, long, radius_km, df_map):
    """
    Dependencies: numpy as np, pandas as pd, geopy.distance
    Input: Latitude (float), Longitude (float), Radius in km (float)
    Output: Average NJOP value based on df_map in the radius (float)
    """
    d_lat = radius_km/110 # let's use upper bound of 110000
    d_long = radius_km/111 # let's assume upper bound of cos(0) and 111000
    df = df_map.copy(deep=True)
    df = df[ (df['latitude'].between(lat-d_lat, lat+d_lat)) & (df['longitude'].between(long-d_long, long+d_long)) ] # bbox
    try:
        df['distance'] = df.apply(lambda x: geopy.distance.distance((lat, long) , (x['latitude'], x['longitude'])).km, axis=1)
    except:
        df['distance'] = np.nan
    df = df[ df['distance'] <= radius_km ]
    return [df['ave_njb'].mean(), df['ave_njb'].count(), df['ave_njb'].min(), df['ave_njb'].max(), df['ave_njb'].std()]

df_lamudi = pd.read_csv('lamudi_house_dataset_cleaned.csv')
df_lamudi = df_lamudi.drop('Unnamed: 0', axis=1) # from the creation of csv
df_lamudi.head()

geo_point_s = sorted(df_lamudi['data_geo_point_s'].astype('str').unique())
print('Addresses count:', len(geo_point_s))
average_njop = []
neighbors = []
counter = 0
min_ave_njop = []
max_ave_njop = []
std_ave_njop = []
for address in geo_point_s:
    counter += 1
    print('Now on address:', counter)
    try:
        address = eval(address)
        sol = get_average_njop_near(address[1], address[0], 1, df_map)
        average_njop.append(sol[0])
        neighbors.append(sol[1])
        min_ave_njop.append(sol[2])
        max_ave_njop.append(sol[3])
        std_ave_njop.append(sol[4])
    except:
        average_njop.append(np.nan)
        neighbors.append(np.nan)
        min_ave_njop.append(np.nan)
        max_ave_njop.append(np.nan)
        std_ave_njop.append(np.nan)
# create new DataFrame of coordinate with NJOP
df_lamudi_coord_njop = pd.DataFrame(data={'data_geo_point_s': geo_point_s, 'njop':average_njop, 'neighbors':neighbors
                                          , 'min_njop': min_ave_njop, 'max_njop': max_ave_njop, 'std_njop': std_ave_njop})

df_lamudi['data_geo_point_s'] = df_lamudi['data_geo_point_s'].astype('str')
df_lamudi = df_lamudi.merge(df_lamudi_coord_njop, on='data_geo_point_s', how='left')
df_lamudi = df_lamudi.drop('data_furnished_s', axis=1)
df_lamudi = df_lamudi.dropna().reset_index(drop=True)

df_lamudi.to_csv('lamudi_njop_dataset.csv', index=False)

## 7.2. Merge Data to Flood Scenarios

**Input Files**:
1. lamudi_njop_dataset.csv
2. flood_0.png
3. flood_1.png
4. flood_2.png
5. flood_3.png
6. flood_4.png
7. flood_5.png
8. flood_6.png 

**Relevant Notebook File**: check_flood.ipynb

**Output File**: lamudi_njop_flood_dataset.csv

In [ ]:
from PIL import Image
import numpy as np
import pandas as pd

def get_pixel(lat, long):
    return [ int(round(160+(478-160)*(long-(106.7351))/(106.91427-((106.7351))))), int(round(145+(654-145)*(lat-(-6.0808))/(-6.36549-((-6.0808))))) ]

def get_flood_scenario(lat, long):
    colors_list = []
    pixel = get_pixel(lat, long)
    flood_0 = 0
    flood_1 = 0
    flood_2 = 0
    flood_3 = 0
    flood_4 = 0
    flood_5 = 0
    flood_6 = 0
    for i in range(7):
        im = Image.open(f"./flood_{i}.png")
        rgb_im = im.convert('RGB')
        for h_px in range(-3,4):
            for v_px in range(-3,4):
                try:
                    colors_list.append( rgb_im.getpixel((pixel[0]+v_px, pixel[1]+h_px)) )
                except:
                    pass
        if i == 0 and (225, 87, 89) in colors_list:
            flood_0 = 1
        if i == 1 and (237, 201, 72) in colors_list:
            flood_1 = 1
        if i == 2 and (78, 121, 167) in colors_list:
            flood_2 = 1
        if i == 3 and (242, 142, 43) in colors_list:
            flood_3 = 1
        if i == 4 and (89, 161, 79) in colors_list:
            flood_4 = 1
        if i == 5 and (176, 122, 161) in colors_list:
            flood_5 = 1
        if i == 6 and (156, 117, 95) in colors_list:
            flood_6 = 1
    return [flood_0, flood_1, flood_2, flood_3, flood_4, flood_5, flood_6]

df = pd.read_csv('lamudi_njop_dataset.csv')

geo_point_s = sorted(df['data_geo_point_s'].astype('str').unique())
print('Addresses count:', len(geo_point_s))
counter = 0
flood_cols = []
for address in geo_point_s:
    counter += 1
    print('Now on address:', counter)
    address = eval(address)
    flood_cols.append(get_flood_scenario(address[1], address[0]))

# create new DataFrame of coordinate with flood scenarios
df_flood = pd.DataFrame(flood_cols, columns=['flood_0', 'flood_1', 'flood_2', 'flood_3', 'flood_4', 'flood_5', 'flood_6'])
df_flood['data_geo_point_s'] = geo_point_s

df['data_geo_point_s'] = df['data_geo_point_s'].astype('str')
df = df.merge(df_flood, on='data_geo_point_s', how='left')

df.to_csv('lamudi_njop_flood_dataset.csv', index=False)

## 7.3. Merge Data to Facilities

**Input Files**:
1. lamudi_njop_flood_dataset.csv
2. facility_latlong.csv

**Relevant Notebook File**: check_facilities.ipynb

**Output File**: final_dataset_unfiltered.csv

In [ ]:
df = pd.read_csv('facilities_latlong.csv')
df_lamudi = pd.read_csv('lamudi_njop_flood_dataset.csv')

# https://stackoverflow.com/questions/1253499/simple-calculations-for-working-with-lat-lon-and-km-distance
# Latitude: 1 deg = 110.574 km
# Longitude: 1 deg = 111.320*cos(latitude) km
def get_facilities_near(lat, long, radius_km, df_map):
    """
    Dependencies: numpy as np, pandas as pd, geopy.distance
    Input: Latitude (float), Longitude (float), Radius in km (float)
    Output: Columns of facility features, that is:
        1. Count Mall facilities near lat and long
        2. Max stars of the Mall facilities near lat and long
        3. Sum of count reviews of the Mall facilities near lat and long
        4. Count Hospital facilities near lat and long
        5. Max stars of the Hospital facilities near lat and long
        6. Sum of count reviews of the Hospital facilities near lat and long
        7. Count Bus Station facilities near lat and long
        9. Sum of count reviews of the Bus Station facilities near lat and long
        8. Max stars of the Train Station facilities near lat and long
        10. Sum of count reviews of the Train Station facilities near lat and long
    """
    d_lat = radius_km/110 # let's use upper bound of 110000
    d_long = radius_km/111 # let's assume upper bound of cos(0) and 111000
    df = df_map.copy(deep=True)
    df = df[ (df['latitude'].between(lat-d_lat, lat+d_lat)) & (df['longitude'].between(long-d_long, long+d_long)) ] # bbox
    try:
        df['distance'] = df.apply(lambda x: geopy.distance.distance((lat, long) , (x['latitude'], x['longitude'])).km, axis=1)
    except:
        df['distance'] = np.nan
    df = df[ df['distance'] <= radius_km ]
    return [
        df[ df['category'] == 'mall']['name'].nunique()
        , df[ df['category'] == 'mall']['stars'].max()
        , df[ df['category'] == 'mall']['count_reviews'].sum()
        , df[ df['category'] == 'hospital']['name'].nunique()
        , df[ df['category'] == 'hospital']['stars'].max()
        , df[ df['category'] == 'hospital']['count_reviews'].sum()
        , df[ df['category'] == 'bus_station']['name'].nunique()
        , df[ df['category'] == 'bus_station']['count_reviews'].sum()
        , df[ df['category'] == 'train_station']['name'].nunique()
        , df[ df['category'] == 'train_station']['count_reviews'].sum()
    ]

geo_point_s = sorted(df_lamudi['data_geo_point_s'].astype('str').unique())
count_mall = []
max_stars_mall = []
sum_reviews_mall = []
count_hospital = []
max_stars_hospital = []
sum_reviews_hospital = []
count_bus_st = []
sum_reviews_bus_st = []
count_train_st = []
sum_reviews_train_st = []
df_map = df
df_map[['latitude', 'longitude']] = df_map[['latitude', 'longitude']].astype('float')
counter = 0
for address in geo_point_s:
    counter += 1
    print('Now on address:', counter)
    try:
        address = eval(address)
        sol = get_facilities_near(address[1], address[0], 2, df_map)
        count_mall.append(sol[0])
        max_stars_mall.append(sol[1])
        sum_reviews_mall.append(sol[2])
        count_hospital.append(sol[3])
        max_stars_hospital.append(sol[4])
        sum_reviews_hospital.append(sol[5])
        count_bus_st.append(sol[6])
        sum_reviews_bus_st.append(sol[7])
        count_train_st.append(sol[8])
        sum_reviews_train_st.append(sol[9])
    except:
        count_mall.append(np.nan)
        max_stars_mall.append(np.nan)
        sum_reviews_mall.append(np.nan)
        count_hospital.append(np.nan)
        max_stars_hospital.append(np.nan)
        sum_reviews_hospital.append(np.nan)
        count_bus_st.append(np.nan)
        sum_reviews_bus_st.append(np.nan)
        count_train_st.append(np.nan)
        sum_reviews_train_st.append(np.nan)

df_facilities = pd.DataFrame(data={'count_mall': count_mall
                                    , 'max_stars_mall': max_stars_mall
                                    , 'sum_reviews_mall': sum_reviews_mall
                                    , 'count_hospital': count_hospital
                                    , 'max_stars_hospital': max_stars_hospital
                                    , 'sum_reviews_hospital': sum_reviews_hospital
                                    , 'count_bus_st': count_bus_st
                                    , 'sum_reviews_bus_st': sum_reviews_bus_st
                                    , 'count_train_st': count_train_st
                                    , 'sum_reviews_train_st': sum_reviews_train_st
                                    , 'data_geo_point_s': geo_point_s})

df_lamudi['data_geo_point_s'] = df_lamudi['data_geo_point_s'].astype('str')
df_lamudi = df_lamudi.merge(df_facilities, on='data_geo_point_s', how='left')
df_lamudi = df_lamudi.dropna().reset_index(drop=True)

df_lamudi.to_csv('final_dataset_unfiltered.csv')

## 8. Feature Engineering and Modelling

**Input File**: final_dataset_unfiltered.csv

**Relevant Notebook File**: feature_engineering_modelling.ipynb

**Output File**: model.pkl

In [ ]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
import random
from math import ceil, floor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error
from feature_engine.encoding import OrdinalEncoder
from xgboost import XGBRegressor
import pickle

pd.pandas.set_option('display.max_columns', None)

df = pd.read_csv('final_dataset_unfiltered.csv')

df['data_geo_point_s'] = df['data_geo_point_s'].astype('str')
df['latitude'] = df['data_geo_point_s'].apply(lambda x: eval(x)[1] if x != 'nan' else None)
df['longitude'] = df['data_geo_point_s'].apply(lambda x: eval(x)[0] if x != 'nan' else None)

# remove kepulauan seribu
df = df[ df['kabupaten'] != 'kepulauan-seribu' ].reset_index(drop=True)
# remove wrong latitude and longitude
df = df[ (df['latitude'].between(-7, -5)) & (df['longitude'].between(106, 108)) ].reset_index(drop=True)

# create floor
df['floor_by_size'] = np.vectorize(ceil)(df['data_building_size_s']/df['data_land_size_s'])
floor_pattern = '(\d+).?(lantai|lt)'
df['floor_pattern'] = df['page_link_s'].apply(lambda x: re.search(floor_pattern, x)[1] if re.search(floor_pattern, x) else 0).astype(int)

def get_floor(floor_pattern, floor_by_size):
    if floor_pattern != 0 and floor_pattern < 5:
        return floor_pattern
    else:
        return floor_by_size

df['floor'] = np.vectorize(get_floor)(df['floor_pattern'], df['floor_by_size'])
df = df.drop(['floor_pattern','floor_by_size'], axis=1)
df[['max_stars_mall', 'max_stars_hospital']] = df[['max_stars_mall', 'max_stars_hospital']].fillna(0)

model_columns = ['data_price_s', 'data_bedrooms_s', 'data_bathrooms_s', 'floor', 'data_land_size_s', 'data_building_size_s', 'njop'
                 , 'neighbors', 'min_njop', 'max_njop', 'std_njop', 'flood_0', 'flood_1', 'flood_2', 'flood_3', 'flood_4'
                 , 'flood_5', 'flood_6', 'count_mall', 'max_stars_mall', 'sum_reviews_mall', 'count_hospital', 'max_stars_hospital'
                 , 'sum_reviews_hospital', 'count_bus_st', 'sum_reviews_bus_st', 'count_train_st', 'sum_reviews_train_st'
                 , 'latitude', 'longitude', 'kabupaten', 'kecamatan']
df = df[model_columns]

# standardize kecamatan from links to the reality in the format of NJOP files
dict_kecamatan = {"ancol": "pademangan", "bendungan-hilir": "tanah-abang", "cempaka-putih-1": "cempaka-putih"
                  , "cibubur": "ciracas", "cipayung-1": "cipayung", "jatinegara-1": "jatinegara", "jelambar": "grogol-petamburan"
                  , "jatinegara-1": "jatinegara", "kemayoran-2": "kemayoran", "kuningan-2": "setiabudi", "setia-budi": "setiabudi"
                  , "pantai-indah-kapuk": "penjaringan", "pluit": "penjaringan", "pondok-indah": "kebayoran-lama"
                  , "pondok-kelapa-1": "duren-sawit", "puri-indah": "kembangan", "rawamangun": "pulo-gadung"
                  , "sawah-besar-1": "sawah-besar", "tanah-sereal": "tambora", "tanjung-duren-utara": "grogol-petamburan"
                  , "thamrin": "menteng"}
df['kecamatan'] = df['kecamatan'].apply(lambda x: dict_kecamatan[x] if x in dict_kecamatan else x).apply(lambda x: str(x).upper().replace('-', ' '))

# clean price_per_size outliers
df = df[
    ((df['data_price_s']/(df['data_land_size_s']+df['data_building_size_s']))/df['njop'])\
    .between(np.percentile(((df['data_price_s']/(df['data_land_size_s']+df['data_building_size_s']))/df['njop']), 1)
    , np.percentile(((df['data_price_s']/(df['data_land_size_s']+df['data_building_size_s']))/df['njop']), 99))
].reset_index(drop=True)

# create model
df['price_per_size'] = df['data_price_s']/(df['data_land_size_s']+df['data_building_size_s'])
df = df.drop('data_price_s', axis=1)
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('price_per_size', axis=1)
    , df['price_per_size']
    , test_size=0.3
    , random_state=42
)
pipe = Pipeline([
    ('categorical_encoder'
     , OrdinalEncoder(encoding_method='ordered'
                      , variables=['kota', 'kecamatan']))
    , ('xgbr', XGBRegressor(random_state=42))
])
param_grid = {
    'xgbr__learning_rate': [0.2]
    , 'xgbr__n_estimators': [120]
    , 'xgbr__max_depth': [6]
    , 'xgbr__min_child_weight': [1]
    , 'xgbr__gamma': [0.5]
    , 'xgbr__reg_alpha': [0.01]
    , 'xgbr__reg_lambda': [5]
}
grid_search_tune = GridSearchCV(pipe, param_grid
                           , cv=5, n_jobs=-1, scoring='neg_mean_absolute_percentage_error')
grid_search_tune.fit(X_train, y_train)
print("Negative Mean Absolute Error Train:", grid_search_tune.score(X_train, y_train))
print("Negative Mean Absolute Error Test:", grid_search_tune.score(X_test, y_test))

# train model with all data
X = df.drop('price_per_size', axis=1)
y = df['price_per_size']
grid_search_tune.fit(X, y)

# store model
with open('model.pkl', 'wb') as f:
    pickle.dump(grid_search_tune, f)
f.close()

# 9. Preparation for Web Apps

**Input**: facility_latlong.csv

**Relevant Notebook File**: user_inputs.ipynb

**Output**: kota_kecamatan_kelurahan_jalan_latlong.csv

In [ ]:
df_njop = pd.read_csv('facility_latlong.csv')
df_njop = df_njop[['kota','kecamatan', 'kelurahan', 'nama_jalan', 'latitude', 'longitude']]
df_njop.to_csv('kota_kecamatan_kelurahan_jalan_latlong.csv')